In [59]:
%load_ext autoreload
%autoreload 2

import os
import preprocessing
from gensim import corpora

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [84]:
import os
from gensim.models.doc2vec import TaggedDocument

def texts_corpus(textdir='data/texts/'):
    files = os.listdir(textdir)
    corpus = []
    source_enum = {}
    for i, file in enumerate(files):
        if '.txt' not in file:
            #print('Skipping index: ' + str(i))
            continue
        with open(textdir + file) as f:
            t = f.read()
            #yield gensim.utils.simple_preprocess(t.replace('\n', ' '))
            corpus.append(
                TaggedDocument(
                    #words=[word.lower() for word in t.replace('\n', ' ').split()],
                    words=preprocessing.doc_preprocessor(t, lemmatize=False),
                    tags=[file.strip('.txt')])
                    #tags=[i])
            )
        source_enum[i] = file.strip('.txt')
            
    return corpus, source_enum
            
train_corpus, source_enum = texts_corpus()

In [ ]:
model = gensim.models.doc2vec.Doc2Vec(vector_size=100, min_count=2, epochs=100)
model.build_vocab(train_corpus)

%time model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)
model.save('data/doc2vec.model')

In [141]:
from pprint import pprint

ivec = model.infer_vector(doc_words=train_corpus[200].words)
#pprint(model.wv.most_similar(positive=[ivec], topn=5))
sims = model.docvecs.most_similar([ivec], topn=len(model.docvecs))

In [142]:
pprint(sims[:5])
pprint(sims[:-6:-1])

[('0910.4177v1', 0.986732006072998),
 ('1110.4669v1', 0.6552059650421143),
 ('1410.7316v1', 0.5830439329147339),
 ('1206.2934v2', 0.5695033073425293),
 ('1202.3217v2', 0.5552552342414856)]
[('1901.01486v1', -0.19823354482650757),
 ('1404.3347v1', -0.15204782783985138),
 ('1711.05289v1', -0.14370861649513245),
 ('0809.4781v3', -0.14349150657653809),
 ('0912.0434v2', -0.13382673263549805)]
